In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


#tracking mlflow
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
import mlflow.sklearn
from time import time
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,hamming_loss, jaccard_score, confusion_matrix, roc_curve, auc)
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import resample

#word2vec, BERT, USE
#word2vec
import gensim
from gensim.models import Word2Vec
# Bert
import os
import transformers
import huggingface_hub
import transformers
from transformers import BertTokenizer, BertModel
os.environ["TF_KERAS"]='1'
#BERT
import torch
#Use 
import tensorflow_hub as hub

#empecher les messages d'erreur
import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Formatage taille cellule pour s'adapter aux graphiques
from IPython.display import display, HTML
display(HTML("<style>.output_scroll { height: auto !important; }</style>"))


df = pd.read_csv("./filtered_questions_clean.csv",sep=',', encoding='utf-8')  
df.head(5)

OSError: [WinError 126] Le module spécifié est introuvable. Error loading "C:\Users\MAUD\anaconda3\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
# Convertir les chaînes de caractères représentant des listes en véritables listes
df['tags'] = df['tags'].apply(ast.literal_eval)

# Binariser les tags
mlb = MultiLabelBinarizer()
df_tags = mlb.fit_transform(df['tags'])

# Séparer les données en jeux d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df[['clean_title', 'clean_body']], df_tags, test_size=0.2, random_state=42)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

In [ ]:
print("Classes binarisées :", mlb.classes_)
number_of_classes = len(mlb.classes_)
print(f"Le nombre de classes uniques (tags) est : {number_of_classes}")

In [ ]:
print("Classes binarisées :", mlb.classes_)
number_of_classes = len(mlb.classes_)
print(f"Le nombre de classes uniques (tags) est : {number_of_classes}")

In [ ]:
# Filtrer les classes ayant moins de 55 occurrences
classes_under_55 = class_occurrences_df[class_occurrences_df['Occurrences'] < 55]

# Afficher les classes et leur nombre d'occurrences
print(classes_under_55)

In [ ]:
# Remplir les valeurs manquantes dans 'clean_title' et 'clean_body'
X_train['clean_title'] = X_train['clean_title'].fillna("")
X_train['clean_body'] = X_train['clean_body'].fillna("")
X_test['clean_title'] = X_test['clean_title'].fillna("")
X_test['clean_body'] = X_test['clean_body'].fillna("")

In [ ]:
def oversample_minority_classes(X_train_vect, y_train, threshold=50, n_samples=100, random_state=42):
    """
    Suréchantillonne les classes minoritaires dans le jeu de données d'entraînement.
    
    Parameters:
        X_train_vect: Matrice des features d'entraînement (sparse matrix)
        y_train: Labels d'entraînement
        threshold: Seuil maximum de présence pour considérer une classe comme minoritaire
        n_samples: Nombre d'échantillons après suréchantillonnage pour les classes minoritaires
        random_state: Random state pour reproductibilité
    
    Returns:
        X_train_oversampled, y_train_oversampled: Données suréchantillonnées
    """
    # Convertir la matrice sparse en DataFrame pour suréchantillonnage
    X_train_df = pd.DataFrame(X_train_vect.toarray())
    y_train_oversampled = y_train.copy()
    X_train_oversampled = X_train_df.copy()

    for i in range(y_train.shape[1]):
        if np.sum(y_train[:, i]) < threshold:  # Suréchantillonner les classes avec moins d'occurrences que le seuil
            indices = np.where(y_train[:, i] == 1)[0]
            
            # Vérifier si indices n'est pas vide
            if len(indices) > 0:
                X_resampled, y_resampled = resample(X_train_df.iloc[indices], y_train[indices], 
                                                    n_samples=n_samples, random_state=random_state)
                X_train_oversampled = pd.concat([X_train_oversampled, X_resampled])
                y_train_oversampled = np.vstack([y_train_oversampled, y_resampled])

    # Convertir en matrice sparse après suréchantillonnage
    X_train_oversampled_sparse = np.asarray(X_train_oversampled)
    
    return X_train_oversampled_sparse, y_train_oversampled

### Word2Vec

In [ ]:
# Préparer les phrases pour Word2Vec
sentences = [text.split() for text in X_train['clean_title'] + " " + X_train['clean_body']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Fonction pour obtenir la moyenne des embeddings Word2Vec pour un texte
def get_w2v_features(text, model):
    words = text.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

# Appliquer aux jeux d'entraînement et de test
X_train_w2v = np.array([get_w2v_features(text, word2vec_model) for text in X_train['clean_title'] + " " + X_train['clean_body']])
X_test_w2v = np.array([get_w2v_features(text, word2vec_model) for text in X_test['clean_title'] + " " + X_test['clean_body']])

In [ ]:
X_train_w2v_oversampled, y_train_w2v_oversampled = oversample_minority_classes(X_train_w2v, y_train, min_occurrences=50, target_samples=100)

### BERT

In [ ]:
# Initialiser le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Fonction pour obtenir les embeddings BERT
def get_bert_features(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Appliquer aux jeux d'entraînement et de test
X_train_bert = np.array([get_bert_features(text, tokenizer, bert_model).flatten() for text in X_train['clean_title'] + " " + X_train['clean_body']])
X_test_bert = np.array([get_bert_features(text, tokenizer, bert_model).flatten() for text in X_test['clean_title'] + " " + X_test['clean_body']])

In [ ]:
X_train_bert_oversampled, y_train_bert_oversampled = oversample_minority_classes(X_train_bert, y_train, min_occurrences=50, target_samples=100)

### USE

In [ ]:
# Charger le modèle USE
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Fonction pour obtenir les embeddings USE
def get_use_features(text, model):
    return model([text]).numpy().flatten()

# Appliquer aux jeux d'entraînement et de test
X_train_use = np.array([get_use_features(text, use_model) for text in X_train['clean_title'] + " " + X_train['clean_body']])
X_test_use = np.array([get_use_features(text, use_model) for text in X_test['clean_title'] + " " + X_test['clean_body']])

In [ ]:
X_train_use_oversampled, y_train_use_oversampled = oversample_minority_classes(X_train_use, y_train, min_occurrences=50, target_samples=100)

## Définir les modèles

In [ ]:
# Définir les modèles
logistic_regression_model = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'))
sgd_classifier_model = OneVsRestClassifier(SGDClassifier(max_iter=1000, random_state=42, class_weight='balanced', loss='log_loss'))

# Définir les paramètres et exécuter les expériences
# Word2Vec + Logistic Regression
params_w2v_lr = {
    "model_type": "Logistic Regression (OneVsRest)",
    "vectorizer": "Word2Vec"
}

# BERT + Logistic Regression
params_bert_lr = {
    "model_type": "Logistic Regression (OneVsRest)",
    "vectorizer": "BERT"
}

# USE + Logistic Regression
params_use_lr = {
    "model_type": "Logistic Regression (OneVsRest)",
    "vectorizer": "USE"
}



# Word2Vec + SGDClassifier
params_w2v_sgd = {
    "model_type": "SGDClassifier (OneVsRest)",
    "vectorizer": "Word2Vec"
}

# BERT + SGDClassifier
params_bert_sgd = {
    "model_type": "SGDClassifier (OneVsRest)",
    "vectorizer": "BERT"
}

# USE + SGDClassifier
params_use_sgd = {
    "model_type": "SGDClassifier (OneVsRest)",
    "vectorizer": "USE"
}

## Fonction d'execution du modèle

In [ ]:
def run_experiment(model, X_train, y_train, X_test, y_test, method_name, vectorizer, reference_model=None):
    """
    Exécute l'entraînement et l'évaluation du modèle.
    
    Parameters:
        model: Le modèle à entraîner.
        X_train: Les features d'entraînement.
        y_train: Les labels d'entraînement.
        X_test: Les features de test.
        y_test: Les labels de test.
        method_name: Le nom de la méthode pour l'identification.
        vectorizer: Le vectoriseur utilisé pour calculer le taux de couverture des mots.
        reference_model: Un modèle de référence pour comparaison (facultatif).

    Returns:
        results: Un dictionnaire contenant l'accuracy, le rapport de classification, le Jaccard score,
                 le taux de couverture, et le modèle entraîné.
    """
    # Entraîner le modèle
    model.fit(X_train, y_train)
    
    # Prédire les probabilités sur l'ensemble de test
    y_pred_proba = model.predict_proba(X_test)
    
    # Initialiser une matrice vide pour les prédictions
    y_pred = np.zeros_like(y_pred_proba, dtype=int)
    
    # Appliquer le seuil de 0.5 et assigner les prédictions
    threshold = 0.5
    for i, proba in enumerate(y_pred_proba):
        indices_above_threshold = np.where(proba >= threshold)[0]
        if len(indices_above_threshold) > 0:
            y_pred[i, indices_above_threshold] = 1
        else:
            top_5_indices = np.argsort(proba)[-5:]
            y_pred[i, top_5_indices] = 1
    
    # Calculer les métriques
    accuracy = accuracy_score(y_test, y_pred)
    jaccard = jaccard_score(y_test, y_pred, average='samples', zero_division=0)
    report = classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0)
    word_coverage = calculate_word_coverage(X_test, vectorizer)
    
    # Retourner les résultats sous forme de dictionnaire
    results = {
        "accuracy": accuracy,
        "jaccard": jaccard,
        "word_coverage": word_coverage,
        "report": report,
        "model": model
    }
    
    print(f"{method_name} Classification Report:")
    print(report)
    print(f"Jaccard Score: {jaccard:.2f}")
    print(f"Word Coverage: {word_coverage:.2f}")
    
    return results

## Fonction pour Cacluler le taux de couverture des mots des questions

In [ ]:
def calculate_word_coverage(X_test, vectorizer):
    """
    Calcule le taux de couverture des mots dans le jeu de test par rapport au vocabulaire du vectorizer.
    
    Parameters:
        X_test: Jeu de test (DataFrame avec des colonnes 'clean_title' et 'clean_body')
        vectorizer: Vectoriseur déjà entraîné sur le jeu d'entraînement
    
    Returns:
        coverage_rate: Le pourcentage de mots du jeu de test couverts par le vocabulaire du vectorizer
    """
    # Remplir les valeurs manquantes avec des chaînes vides
    X_test = X_test.fillna("")
    
    # Nombre total de mots uniques dans le vocabulaire (déjà appris lors du fit sur X_train)
    total_vocab = len(vectorizer.get_feature_names_out())
    
    # Transformer les questions de test pour obtenir les mots utilisés dans X_test
    X_test_transformed = vectorizer.transform(X_test['clean_title'] + " " + X_test['clean_body'])
    
    # Compter le nombre de mots uniques utilisés dans le jeu de test
    word_usage = (X_test_transformed > 0).sum(axis=0)
    covered_words = (word_usage > 0).sum()
    
    # Calculer le taux de couverture par rapport au vocabulaire total
    coverage_rate = covered_words / total_vocab
    return coverage_rate

## Fonction d'enregistrement des logs MLFlow

In [ ]:
def log_experiment_to_mlflow(results, params, method_name, threshold=0.5, model_name="My_Model"):
    """
    Enregistre les résultats de l'expérimentation dans MLflow et enregistre le modèle dans le Model Registry.
    
    Parameters:
        results: Un dictionnaire contenant l'accuracy, le rapport de classification, et le modèle entraîné.
        params: Les paramètres du modèle et de la vectorisation à enregistrer dans MLflow.
        method_name: Le nom de la méthode pour l'identification.
        threshold: Le seuil utilisé pour déterminer les classes avec predict_proba.
        model_name: Le nom sous lequel le modèle sera enregistré dans le Model Registry.
    """
    with mlflow.start_run(run_name=method_name) as run:
        # Loguer les paramètres
        mlflow.log_params(params)
        
        # Loguer les métriques
        mlflow.log_metric("accuracy", results["accuracy"])
        mlflow.log_metric("jaccard_score", results["jaccard"])
        mlflow.log_metric("word_coverage", results["word_coverage"])
        
        # Loguer le seuil utilisé pour les prédictions avec predict_proba
        mlflow.log_param("threshold", threshold)
        
        # Loguer la logique d'application du seuil et de sélection des 5 meilleures probabilités
        mlflow.log_param("prediction_logic", "Threshold at 0.5, top 5 if none above threshold")
        
        # Loguer le rapport de classification comme un fichier texte
        mlflow.log_text(results["report"], f"classification_report_{method_name}.txt")
        
        # Loguer le modèle comme un artefact MLflow
        mlflow.sklearn.log_model(results["model"], f"{method_name}_model")

        # Enregistrement du modèle dans le registre de modèles
        model_uri = f"runs:/{run.info.run_id}/{method_name}_model"
        try:
            registered_model = mlflow.register_model(model_uri, model_name)
            print(f"Le modèle '{method_name}' a été enregistré dans le Model Registry avec le nom '{model_name}'.")
        
        except Exception as e:
            print(f"Erreur lors de l'enregistrement dans le Model Registry: {e}")

## Execution des modèles

In [ ]:
#W2V + Logistic Regression
results_w2v_lr = run_experiment(logistic_regression_model, X_train_w2v_oversampled, y_train_w2v_oversampled, X_test_w2v, y_test, "w2v_logistic_regression", vectorizer=word2vec_model)
log_experiment_to_mlflow(results_w2v_lr, params_w2v_lr, "w2v_logistic_regression")

#BERT + Logistic Regression
results_bert_lr = run_experiment(logistic_regression_model, X_train_bert_oversampled, y_train_bert_oversampled, X_test_bert, y_test, "bert_logistic_regression", vectorizer=bert_model)
log_experiment_to_mlflow(results_bert_lr, params_bert_lr, "bert_logistic_regression")

#USE + Logistic Regression
results_use_lr = run_experiment(logistic_regression_model, X_train_use_oversampled, y_train_use_oversampled, X_test_use, y_test, "use_logistic_regression", vectorizer=use_model)
log_experiment_to_mlflow(results_use_lr, params_use_lr, "use_logistic_regression")



#W2V + SGDClassifier
results_w2v_sgd = run_experiment(sgd_classifier_model, X_train_w2v_oversampled, y_train_w2v_oversampled, X_test_w2v, y_test, "w2v_sgd_classifier", vectorizer=word2vec_model)
log_experiment_to_mlflow(results_w2v_sgd, params_w2v_sgd, "w2v_sgd_classifier")

#BERT + SGDClassifier
results_bert_sgd = run_experiment(sgd_classifier_model, X_train_bert_oversampled, y_train_bert_oversampled, X_test_bert, y_test, "bert_sgd_classifier", vectorizer=bert_model)
log_experiment_to_mlflow(results_bert_sgd, params_bert_sgd, "bert_sgd_classifier")

#BERT + SGDClassifier
results_use_sgd = run_experiment(sgd_classifier_model, X_train_use_oversampled, y_train_use_oversampled, X_test_use, y_test, "use_sgd_classifier", vectorizer=use_model)
log_experiment_to_mlflow(results_use_sgd, params_use_sgd, "use_sgd_classifier")